In [1]:
import json
import string
import numpy as np

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
buck_t1=8  #threshold bucket1
max_out_len=12

In [3]:
#upload dictionaries

with open('dict_w2i_chatbot.json') as json_file: 
    dict_w2i = json.load(json_file) 

with open('dict_i2w_chatbot.json') as json_file: 
    dict_i2w = json.load(json_file) 

with open('contractions_dict.json') as json_file: 
    contractions_dict = json.load(json_file) 

In [4]:
one_hot_len=len(dict_w2i)

In [20]:
#upload models

encoder_model= load_model('models/encoder_inf_b1_nobucket_noval.h5')
decoder_model= load_model('models/decoder_inf_b1_nobucket_noval.h5')

In [21]:
#preprocessing input string
def exp_remPunt(sent):
    '''
    clean the input sentence
    - set to lower
    - remove contract form
    - remove punctuation
    '''
    clean_sent=[]
    table = str.maketrans(dict.fromkeys(string.punctuation))
    remove_digits = str.maketrans('', '', string.digits)
    sent=sent.lower()
    
    for word in sent.split():
        if word in contractions_dict:
            sent = sent.replace(word, contractions_dict[word])  #expand

    sent = sent.translate(remove_digits)
    clean_sent.append(sent.translate(table).lower()) #remove punt and set to lower
    
    return clean_sent

In [22]:
def string_to_seq(l_sent):
    '''
    input list string 
    '''
    emb_str=[]
    for l in l_sent:
        for word in l.split():
            if (word in dict_w2i):
                emb_str.append(dict_w2i[word])

   
    emb_str = pad_sequences([emb_str],8, padding='post')
    return np.flip(emb_str)  #reverse and return

In [23]:
def chat_loop(sentence):
    
    stop_model=False #iteration condition
    
    
    
    sentence=exp_remPunt(sentence)
    if sentence=='quitbot':
        stop_model=True
    sentence=string_to_seq(sentence)
    
    #define init variables
    target_seq=np.zeros((1,1))

    target_seq[0,0]=dict_w2i['<sos>']  #target seq must be init with start word

    
    answer_seq=[]  #the final answer will be a sequence of numbers

    #encode the input
    states_= encoder_model.predict(sentence)
    while not stop_model:

        decoder_output,state_h,state_c=decoder_model.predict([target_seq]+states_)
        #get sampled word
        sampled_word=np.argmax(decoder_output[0,-1,:])

        #check iteration condition: stop model if eos or len>output len
        if sampled_word==dict_w2i['<eos>'] or len(answer_seq)>=max_out_len:
            stop_model=True
            continue
        else:
            answer_seq.append(sampled_word)

        #update target seq for next iteration
        target_seq=np.zeros((1,1))
        target_seq[0,0]=sampled_word #last word

        #update states
        states_=[state_h,state_c]  
    #print(answer_seq)
    #answer_seq
    mm="BOT:"
    for i in answer_seq:
        mm=mm+" " +dict_i2w[str(i)]
    print(mm)

#print(f'You entered {sentence}')

In [ ]:
print("Type \"quitbot\" to exit. Let\'s start! \n \n")
stop_condition=False

while not stop_condition:
    sentence = input("YOU: ")
    if sentence=='quitbot':
        print("good bye")
        stop_condition=True
        continue
    chat_loop(sentence)

Type "quitbot" to exit. Let's start! 
 

YOU: Good morning
BOT: but i have not
YOU: good night
BOT: good night
YOU: do you know me?
BOT: yes
YOU: really?
BOT: i hope im not being too forward
YOU: don't worry, it's okay
BOT: what do you mean
YOU: you are not bothering
BOT: youre just are you are so
YOU: be gentle and kind
BOT: i am so sorry
YOU: do you like a colour?
BOT: oh yes i really like it
YOU: which one
BOT: any one i need to sharpen this pencil
YOU: good bye
BOT: what is it
YOU: it's a way to finish this conversation
BOT: how many of them tonight
YOU: conversations?
BOT: so what else was it
